In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import ssl
from sqlalchemy import create_engine, inspect

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
movie_data = "../Resources/movies.csv"

movies_df = pd.read_csv(movie_data)

movies_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


In [3]:
# Cleaned Dataset

movies_cleaned_df = movies_df.dropna()

movies_cleaned_df['votes'] = movies_cleaned_df['votes'].astype(int)
movies_cleaned_df['budget'] = movies_cleaned_df['budget'].astype(int)
movies_cleaned_df['gross'] = movies_cleaned_df['gross'].astype(int)
movies_cleaned_df['runtime'] = movies_cleaned_df['runtime'].astype(int)

movies_cleaned_df

/Users/calicritic93/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/calicritic93/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/calicritic93/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772,Warner Bros.,146
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106,Columbia Pictures,104
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067,Lucasfilm,124
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539,Paramount Pictures,88
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344,Orion Pictures,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,Bad Boys for Life,R,Action,2020,"January 17, 2020 (United States)",6.6,140000,Adil El Arbi,Peter Craig,Will Smith,United States,90000000,426505244,Columbia Pictures,124
7649,Sonic the Hedgehog,PG,Action,2020,"February 14, 2020 (United States)",6.5,102000,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000,319715683,Paramount Pictures,99
7650,Dolittle,PG,Adventure,2020,"January 17, 2020 (United States)",5.6,53000,Stephen Gaghan,Stephen Gaghan,Robert Downey Jr.,United States,175000000,245487753,Universal Pictures,101
7651,The Call of the Wild,PG,Adventure,2020,"February 21, 2020 (United States)",6.8,42000,Chris Sanders,Michael Green,Harrison Ford,Canada,135000000,111105497,20th Century Studios,100


In [4]:
movies_cleaned_df.to_csv('movies_cleaned.csv')

In [20]:
# Model Dataset

movies_model_df = movies_cleaned_df.drop(columns=['name','released','votes','writer','year','country'])

movies_model_df

,rating,genre,score,director,star,budget,gross,company,runtime
0,R,Drama,8.4,Stanley Kubrick,Jack Nicholson,19000000,46998772,Warner Bros.,146
1,R,Adventure,5.8,Randal Kleiser,Brooke Shields,4500000,58853106,Columbia Pictures,104
2,PG,Action,8.7,Irvin Kershner,Mark Hamill,18000000,538375067,Lucasfilm,124
3,PG,Comedy,7.7,Jim Abrahams,Robert Hays,3500000,83453539,Paramount Pictures,88
4,R,Comedy,7.3,Harold Ramis,Chevy Chase,6000000,39846344,Orion Pictures,98
...,...,...,...,...,...,...,...,...,...
7648,R,Action,6.6,Adil El Arbi,Will Smith,90000000,426505244,Columbia Pictures,124
7649,PG,Action,6.5,Jeff Fowler,Ben Schwartz,85000000,319715683,Paramount Pictures,99
7650,PG,Adventure,5.6,Stephen Gaghan,Robert Downey Jr.,175000000,245487753,Universal Pictures,101
7651,PG,Adventure,6.8,Chris Sanders,Harrison Ford,135000000,111105497,20th Century Studios,100


In [21]:
movies_model_df.to_csv('movies_model.csv')

In [22]:
# Database Connection
engine = create_engine("sqlite:///Movies.sqlite")
conn = engine.connect()

In [23]:
# Session To Connect DB
from sqlalchemy.orm import Session
session = Session(bind=engine)